## Ejercicios de pair programming 23 enero: Anova

En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.

Hasta ahora habéis estado evaluando las características de vuestro set de datos y habéis hecho una gran exploración, es el momento de hacer vuestro primer ANOVA! En el ejercicio de hoy tendréis que hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.

 📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer cuando nos encontramos en esta situación.

In [26]:
# Importamos las librerias necesarias

# Tratamiento de datos
import numpy as np
import pandas as pd
import random 

# Estadísticos
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None 

In [27]:
# Abrimos nuestro DataFrame
df = pd.read_csv("archive/estandarizado.csv", index_col = 0)
df.head(2)

,Country,Region,Happiness Rank,Happiness Score,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,-0.993631,7.587,0.667797,1.184471,0.817892,1.142818,2.674027,0.488939,0.450617
1,Iceland,Western Europe,-0.980892,7.561,0.519256,1.336192,0.835310,0.976182,0.301121,1.360830,0.722398


In [28]:
columnas = {col: col.replace(" ", "_").lower() for col in df.columns}
df.rename(columns = columnas, inplace=True)
df.head()

,country,region,happiness_rank,happiness_score,economy_(gdp_per_capita),family,health_(life_expectancy),freedom,trust_(government_corruption),generosity,dystopia_residual
0,Switzerland,Western Europe,-0.993631,7.587,0.667797,1.184471,0.817892,1.142818,2.674027,0.488939,0.450617
1,Iceland,Western Europe,-0.980892,7.561,0.519256,1.336192,0.835310,0.976182,0.301121,1.360830,0.722398
2,Denmark,Western Europe,-0.968153,7.527,0.555780,1.216329,0.636395,1.069507,3.217869,0.767717,0.413316
3,Norway,Western Europe,-0.955414,7.522,0.766346,1.131058,0.665118,1.161655,2.207255,0.802712,0.373969
4,Canada,North America,-0.942675,7.427,0.557057,1.107057,0.720608,0.995200,1.904941,1.497125,0.354023


In [29]:
df.rename(columns={"economy_(gdp_per_capita)" : "economy", "health_(life_expectancy)" : "health", "trust_(government_corruption)" : "trust"}, inplace=True)

In [30]:
df.head(2)

,country,region,happiness_rank,happiness_score,economy,family,health,freedom,trust,generosity,dystopia_residual
0,Switzerland,Western Europe,-0.993631,7.587,0.667797,1.184471,0.817892,1.142818,2.674027,0.488939,0.450617
1,Iceland,Western Europe,-0.980892,7.561,0.519256,1.336192,0.835310,0.976182,0.301121,1.360830,0.722398


In [32]:
lm = ols('happiness_score ~ happiness_rank + economy + family + health + freedom + trust + generosity + dystopia_residual', data = df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
happiness_rank,1.0,404.198421,4.041984e+02,4.769583e+09,0.000000e+00
economy,1.0,0.000516,5.161836e-04,6.091020e+03,4.678323e-204
family,1.0,0.097087,9.708682e-02,1.145634e+06,0.000000e+00
health,1.0,0.015172,1.517150e-02,1.790253e+05,0.000000e+00
freedom,1.0,0.091493,9.149324e-02,1.079630e+06,0.000000e+00
trust,1.0,0.162572,1.625716e-01,1.918361e+06,0.000000e+00
generosity,1.0,0.054360,5.435999e-02,6.414534e+05,0.000000e+00
dystopia_residual,1.0,4.551473,4.551473e+00,5.370784e+07,0.000000e+00
Residual,306.0,0.000026,8.474503e-08,NaN,NaN


In [33]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        happiness_score   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 6.035e+08
Date:                Mon, 23 Jan 2023   Prob (F-statistic):               0.00
Time:                        10:32:47   Log-Likelihood:                 2122.3
No. Observations:                 315   AIC:                            -4227.
Df Residuals:                     306   BIC:                            -4193.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             5.5005    2.5e-05    2.2e+05      0.000       5.500       5.501
happiness_rank       -0.0002      0.000     -0.821      0.412      -0.001       0.000
economy               0.6341   9.71e-05   6528.747      0.000       0.634       0.634
family                0.3474   5.15e-05   6749.671      0.000       0.347       0.348
health                0.3679   6.86e-05   5366.674      0.000       0.368       0.368
freedom               0.2208   4.43e-05   4979.697      0.000       0.221       0.221
trust                 0.1173   2.37e-05   4957.897      0.000       0.117       0.117
generosity            0.1600   2.97e-05   5396.211      0.000       0.160       0.160
dystopia_residual     0.6793   9.27e-05   7328.564      0.000       0.679       0.679
==============================================================================
Omnibus:                      146.825   Durbin-Watson:                   2.060
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               18.217
Skew:                          -0.053   Prob(JB):                     0.000111
Kurtosis:                       1.827   Cond. No.                         25.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

El **R-squared** tiene un valor de 1, lo que implica que nuestra variable respuesta depende al 100% de las variables predictoras.

Teniendo el cuenta el **p-valor**, no tendremos en cuenta la columna hapiness rank, ya que no tiene ninguna influencia en la respuesta.


In [37]:
df.to_csv("archive/anova.csv", index=False)